In [ ]:
import glob
import numpy as np
import pandas as pd

from nltk.tokenize import TweetTokenizer, ToktokTokenizer
import re

%matplotlib inline

In [14]:
pd.set_option('display.max_columns', len(df.columns))

In [273]:
random_flood = glob.glob('data/opendatascience Slack export Feb 8 2018/_random_flood/*.json')

In [269]:
deep_learning = glob.glob('data/opendatascience Slack export Feb 8 2018/')

In [353]:
proj_dsbowl_2017 = glob.glob('data/opendatascience Slack export Feb 8 2018/proj_dsbowl_2017/*.json')

In [356]:
proj_ods_introspect = glob.glob('data/opendatascience Slack export Feb 8 2018/proj_ods_introspect/*.json')

In [416]:
proj_ = glob.glob('data/opendatascience Slack export Feb 8 2018/proj_*/*.json')

In [504]:
all_ = glob.glob('data/opendatascience Slack export Feb 8 2018/*/*.json')

In [413]:
df = pd.concat(map(pd.read_json, deep_learning))

In [505]:
df = pd.concat(map(pd.read_json, all_))

In [597]:
def remove_links(text):
    return re.sub('<?\s?https?.+>?', '', text)

def substitute(text):
    text = re.sub('\:([_\w\d]+)\:', '_\\1_', text)
    text = re.sub('<[@#](\w)([\w\d]+)\|?[_\w\d]+>', '', text)
    text = re.sub('\s?\d+\s?', ' ', text)
    return text

def filter_tokens(tokens):
    return filter(lambda token: not re.match('[\~,:<>\+\-\*`\'"\^\@\/]{1,}|\.{2,}', token), tokens)

In [507]:
df['ts'] = df['ts'].astype(int).apply(pd.datetime.fromtimestamp)

In [508]:
df['period'] = df.ts.dt.round(pd.tseries.frequencies.Second(10))

In [509]:
texts = df[df.subtype.isnull()]['text'] #.groupby(['user', 'period'])['text'].apply('.'.join)

In [599]:
corpus = texts[texts.apply(lambda text: isinstance(text, str))].sample(300000) \
    .apply(lambda text: tokenizer.tokenize(substitute(text)))

In [623]:
words = np.concatenate(corpus.values)
counts = pd.value_counts(words)
unk = dict(counts[counts <= 5])

In [602]:
def replace_unk(tokens):
    return np.array(['_unk_' if token in unk else token for token in tokens])

In [624]:
corpus1 = corpus[(corpus.apply(len) >= 2) & (corpus.apply(len) <= 10)] \
    .apply(lambda tokens: replace_unk(filter_tokens(tokens)))
corpus2 = corpus1[corpus1.apply(lambda tokens: len(tokens[tokens == '_unk_']) < 2)] \
    .apply(lambda tokens: remove_links(' '.join(tokens)))

In [625]:
corpus2 = corpus2[corpus2.str.len() > 0]
corpus2 = corpus2.str.strip()
len(corpus2)

267842

In [626]:
import csv
corpus2.to_frame().to_csv('data/dota.csv', sep='\t', index=False, header=False, quoting=csv.QUOTE_NONE, escapechar='\\')

In [614]:
dota = pd.read_csv('data/unk_time_token_dotadata10.csv', header=None)

In [622]:
corpus = dota[0].str.split(' ')